# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests as req

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dompu,-8.5356,118.4622,25.05,97,88,0.80,ID,2024-04-24 17:55:20
1,1,ust-nera,64.5664,143.2378,1.93,93,99,2.45,RU,2024-04-24 17:55:22
2,2,bukhara,39.7676,64.4231,22.80,32,79,4.54,UZ,2024-04-24 17:55:23
3,3,atafu village,39.7676,64.4231,22.80,32,79,4.54,UZ,2024-04-24 17:55:23
4,4,puerto baquerizo moreno,-0.9026,-89.6102,25.50,85,17,3.14,EC,2024-04-24 17:55:27


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plt_city_data = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City",
    fill_alpha = .5
)

# Display the map
plt_city_data

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
vacation_weather_df = city_data_df.copy()

# Ideal temperature Humidity Cloudiness Wind Speed
temp_high = 95 # <- degrees fahrenheit
temp_low = 70 # <- degrees fahrenheit
humid_range = range(40, 50)

vacation_weather_df = vacation_weather_df.loc[vacation_weather_df['Max Temp'] >= ((temp_low-32)*5/9) ]
vacation_weather_df = vacation_weather_df.loc[vacation_weather_df['Max Temp'] < ((temp_high-32)*5/9)]
vacation_weather_df = vacation_weather_df.loc[vacation_weather_df['Humidity'].isin(humid_range)]

# Drop any rows with null values
vacation_weather_df = vacation_weather_df.dropna(axis=0, how='any') 

# Display sample data
vacation_weather_df.sample()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
192,192,carnarvon,-24.8826,113.6576,23.5,42,76,8.38,AU,2024-04-24 18:06:20


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.sample()

,City,Country,Lat,Lng,Humidity,Hotel Name
487,siavonga,ZM,-16.5365,28.7102,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "hotel":"categories=accommodation.hotel",
    "filter":"",
    "bias":"",
    "key":f"apiKey={geoapify_key}"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    hLat = hotel_df.loc[index]['Lat']
    hLon = hotel_df.loc[index]['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"filter=circle:{str(hLon)},{str(hLat)},{str(radius)}"
    params["bias"] = f"bias=proximity:{str(hLon)},{str(hLat)}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    hotel = params["hotel"]
    filter = params["filter"]
    bias = params["bias"]
    key = params["key"]
    name_address = f"{base_url}?{hotel}&{filter}&{bias}&{key}" 
    name_address = req.get(name_address).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.sample()


Starting hotel search
lander - nearest hotel: Holiday Lodge
betul - nearest hotel: Hotel Abhashree
carnarvon - nearest hotel: No hotel found
bolivar - nearest hotel: Hotel Eddy
edd - nearest hotel: No hotel found
snyder - nearest hotel: Days Inn
gateway - nearest hotel: Four Points by Sheraton
siavonga - nearest hotel: Lake Safari Lodge Annex
gizo - nearest hotel: فندق العربي
baihar - nearest hotel: Vikash Mehre


,City,Country,Lat,Lng,Humidity,Hotel Name
192,carnarvon,AU,-24.8826,113.6576,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
plt_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = ['Country', 'Hotel Name'],
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2.0,
    color = "City",
    fill_alpha = .5
)

# Display the map
plt_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)